In [48]:
import pandas as pd
import numpy as np
import math
import math
from pathlib import Path

from tournament import Tournament, Participant, Match

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
OPTION_DEV = "-c search_path=dev"
OPTION_PROD = "-c search_path=prod"
GAME_ID = 406  # 4 Team 2021
GAME_ID = 359  # 6 Team 2016
GAME_ID = 273  # 8 team 2012

In [49]:
league_settings_query = f"SELECT game_id, league_id, max_teams, num_playoff_consolation_teams, num_playoff_teams, playoff_start_week  from dev.league_settings where game_id = '{GAME_ID}'"
league_settings = (
    DatabaseCursor(PATH, options=OPTION_DEV)
    .copy_data_from_postgres(league_settings_query)
    .drop_duplicates()
)

meta_query = "SELECT game_id, league_id, end_week FROM dev.league_metadata"
meta = DatabaseCursor(PATH).copy_data_from_postgres(meta_query).drop_duplicates()

league_settings = league_settings.merge(
    meta,
    how="inner",
    left_on=["game_id", "league_id"],
    right_on=["game_id", "league_id"],
)

num_playoff_teams = league_settings["num_playoff_teams"].values[0]
num_conso_teams = league_settings["num_playoff_consolation_teams"].values[0]
max_teams = league_settings["max_teams"].values[0]
num_toliet_teams = (max_teams-num_playoff_teams if num_playoff_teams >= (max_teams/2) else num_playoff_teams) if num_conso_teams != 0 else False
playoff_start_week = league_settings["playoff_start_week"].values[0]
playoff_end_week = league_settings["end_week"].values[0]
playoff_weeks = range(playoff_start_week, playoff_end_week + 1)

matchup_board_query = f"SELECT * FROM prod.playoff_board_{GAME_ID}"
matchup_board = DatabaseCursor(PATH, options=OPTION_PROD).copy_data_from_postgres(matchup_board_query).drop_duplicates()

playoff_teams = list(
    matchup_board["team_name"][
        matchup_board["reg_season_rank"] <= num_playoff_teams
    ].unique()
)
if num_toliet_teams != 0:
    toilet_teams = list(
        matchup_board["team_name"][
            matchup_board["reg_season_rank"] > (max_teams - num_toliet_teams)
        ].unique()
    )
else:
    toilet_teams = False

matchup_board.set_index(matchup_board["team_key"], drop=True, inplace=True)
print(num_playoff_teams)
print(num_toliet_teams)
league_settings

Successfully pulled: SELECT game_id, league_id, max_teams, num_playoff_consolation_teams, num_playoff_teams, playoff_start_week  from dev.league_settings where game_id = '359'
Successfully pulled: SELECT game_id, league_id, end_week FROM dev.league_metadata
Successfully pulled: SELECT * FROM prod.playoff_board_359
6
4


,game_id,league_id,max_teams,num_playoff_consolation_teams,num_playoff_teams,playoff_start_week,end_week
0,359,185373,10,6,6,14,16


In [50]:
print(playoff_teams)
print(toilet_teams)

playoff_bracket = Tournament(playoff_teams)
num_of_rounds = int(math.ceil(math.log2(len(playoff_teams))))
for round_ in range(num_of_rounds):
    for match in playoff_bracket.get_active_matches():
        right_comp = match.get_participants()[0].get_competitor()
        right_score = matchup_board["team_pts"][
            (matchup_board["team_name"] == right_comp)
            & (matchup_board["week"] == playoff_start_week + round_)
        ].values[0]
        left_comp = match.get_participants()[1].get_competitor()
        left_score = matchup_board["team_pts"][
            (matchup_board["team_name"] == left_comp)
            & (matchup_board["week"] == playoff_start_week + round_)
        ].values[0]
        if right_score > left_score:
            match.set_winner(right_comp)
        elif right_score < left_score:
            match.set_winner(left_comp)
print(playoff_bracket.get_final())

if toilet_teams:
    toilet_bracket = Tournament(toilet_teams)
    num_of_rounds = int(math.ceil(math.log2(len(toilet_teams))))
    for round_ in range(num_of_rounds):
        for match in toilet_bracket.get_active_matches():
            right_comp = match.get_participants()[0].get_competitor()
            right_score = matchup_board["team_pts"][
                (matchup_board["team_name"] == right_comp)
                & (matchup_board["week"] == playoff_start_week + round_)
            ].values[0]
            left_comp = match.get_participants()[1].get_competitor()
            left_score = matchup_board["team_pts"][
                (matchup_board["team_name"] == left_comp)
                & (matchup_board["week"] == playoff_start_week + round_)
            ].values[0]
            print(f"{right_comp}: {right_score} vs {left_comp}: {left_score}")
            if right_score > left_score:
                match.set_winner(right_comp)
            elif right_score < left_score:
                match.set_winner(left_comp)
            elif right_score == left_score:
                match.set_winner(left_comp)
    print(toilet_bracket.get_final())

['Lock Jaw Blowy', 'Push Ups in Prison', 'Breesus Christ', 'Dez Nuts', 'Stairway to Evans', "2 Gurley's 1 Cup"]
['Arian Nation', "Brady didn't do it", 'Deckers Barn animals', 'Adele Beckham Jr.']
Breesus Christ: 72.18 vs 2 Gurley's 1 Cup: 111.6
Dez Nuts: 85.5 vs Stairway to Evans: 103.24
Lock Jaw Blowy: 85.58 vs Stairway to Evans: 94.42
Push Ups in Prison: 122.2 vs 2 Gurley's 1 Cup: 117.82
Breesus Christ: 103.56 vs Dez Nuts: 106.84
Stairway to Evans: 112.26 vs Push Ups in Prison: 103.52
Lock Jaw Blowy: 131.28 vs 2 Gurley's 1 Cup: 101.7
{'1': 'Stairway to Evans', '2': 'Push Ups in Prison', '3': 'Lock Jaw Blowy', '4': "2 Gurley's 1 Cup", '5': 'Dez Nuts', '6': 'Breesus Christ'}
Arian Nation: 92.22 vs Adele Beckham Jr.: 127.86
Brady didn't do it: 61.94 vs Deckers Barn animals: 67.28
Adele Beckham Jr.: 84.54 vs Deckers Barn animals: 72.02
Arian Nation: 126.72 vs Brady didn't do it: 79.2
{'1': 'Adele Beckham Jr.', '2': 'Deckers Barn animals', '3': 'Arian Nation', '4': "Brady didn't do it"}


In [ ]:
# # def test_tourney():

# __matches = []
# next_higher_power_of_two = int(
#     math.pow(2, math.ceil(math.log2(len(playoff_teams))))
# )
# winners_number_of_byes = next_higher_power_of_two - len(playoff_teams)
# incoming_participants = list(map(Participant, playoff_teams))
# incoming_participants.extend([None] * winners_number_of_byes)
# num_of_rounds = int(math.ceil(math.log2(len(incoming_participants))))

# round_ = 1
# while round_ <= num_of_rounds:
#     if round_ == 1:
#         half_length = int(len(incoming_participants) / 2)
#         first = incoming_participants[0:half_length]
#         last = incoming_participants[half_length:]
#         last.reverse()
#         round_1_winners = []
#         round_1_losers = []
#         for participant_pair in zip(first, last):
#             if participant_pair[1] is None:
#                 round_1_winners.append(participant_pair[0])
#             elif participant_pair[0] is None:
#                 round_1_winners.append(participant_pair[1])
#             else:
#                 match = Match(participant_pair[0], participant_pair[1])
#                 round_1_winners.append(match.get_winner_participant())
#                 round_1_losers.append(match.get_loser_participant())
#                 __matches.append(match)
#         next_round = [round_1_winners, round_1_losers]
#         round_ += 1

#     elif round_ > 1:
#         while True:
#             if len(next_round[0]) > 1:
#                 print('Begin', next_round)
#                 bracket = next_round[0]
#                 print('Bracket', bracket)
#                 half_length = int(len(bracket) / 2)
#                 first = bracket[0:half_length]
#                 last = bracket[half_length:]
#                 last.reverse()
#                 next_round_winners = []
#                 next_round_losers = []
#                 for participant_pair in zip(first, last):
#                     if participant_pair[1] is None:
#                         next_round_winners.append(participant_pair[0])
#                     elif participant_pair[0] is None:
#                         next_round_winners.append(participant_pair[1])
#                     else:
#                         match = Match(participant_pair[0], participant_pair[1])
#                         next_round_winners.append(match.get_winner_participant())
#                         next_round_losers.append(match.get_loser_participant())
#                         __matches.append(match)

                
#                 if winners_number_of_byes > 0 and len(bracket) > len(next_round[1]):
#                     next_round.insert(0, next_round_losers)
#                     next_round.insert(0, next_round_winners)
#                     winners_number_of_byes = 0
                    
#                 else:
#                     next_round.append(next_round_winners)
#                     next_round.append(next_round_losers)

#                 try:
#                     next_round.remove(bracket)
#                 except:
#                     continue
                
#                 next_round = [ele for ele in next_round if ele != []]
#                 print('Start New', next_round)
#                 if len(next_round) == len(playoff_teams):
#                     break
#             round_ += 1

#     # return next_round, __matches, competitors_list